In [1]:
import salabim_plus as sim_plus
# import salabim_plus_raw as sim_plus
import salabim as sim
import part_a, part_b, part_c
import misc_tools
import datetime

now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
runtime = 10080
 
with open('data/output_'+now+'.txt', 'w') as out:
    
    # step 1: create the simulation environment
#     env = sim_plus.EnvironmentPlus(trace=out)
    env = sim_plus.Environment(trace=out)
    
    # step 2: create the machines in the simulation environment
    assembly_bench_1 = sim_plus.Machine(var_name='assembly_bench_1', env=env)
    assembly_bench_2 = sim_plus.Machine(var_name='assembly_bench_2', env=env)
    
    machine_1 = sim_plus.Machine(var_name='machine_1', env=env)
    machine_2 = sim_plus.Machine(var_name='machine_2', env=env)
    machine_3 = sim_plus.Machine(var_name='machine_3', env=env)
    
    # step 3: make machine groupings for common processes
    assembly_bench = sim_plus.MachineGroup(var_name='assembly_bench', env=env,
                                           machines=[assembly_bench_1,
                                                     assembly_bench_2])
    common_process = sim_plus.MachineGroup(var_name='common_process', env=env,
                                           machines=[machine_1, machine_3])
    
    # step 4: create the workers in the simulation environment
    assembler = sim_plus.Worker(var_name='assembler', env=env, capacity=2)
    technician = sim_plus.Worker(var_name='technician', env=env, capacity=2)
    
    # step 5: make the shift schedules for the workers
    shift_schedule = misc_tools.make_shifts(shift_duration=8*60,
                                            off_days=['saturday','sunday'])
    
    assembler_shift = sim_plus.ShiftController(worker=assembler,
                                               env=env,
                                               start_time=480,
                                               shifts=shift_schedule, 
                                               shift_type='pattern' )
    technician_shift = sim_plus.ShiftController(worker=technician,
                                               env=env,
                                               start_time=480,
                                               shifts=shift_schedule, 
                                               shift_type='pattern' )
    
    # step 6: point to the step creation function in each entity .py file
    part_a_steps_func = part_a.create_routing
    part_b_steps_func = part_b.create_routing
    part_c_steps_func = part_c.create_routing
    
    # step 7: make all of the generators that determine when entities are made
    part_a_gen = sim_plus.EntityGenerator(var_name='part_a', 
                                         steps_func=part_a_steps_func, 
                                         env=env,
                                         arrival_type='ordered')
    part_b_gen = sim_plus.EntityGenerator(var_name='part_b', 
                                         steps_func=part_b_steps_func, 
                                         env=env,
                                         arrival_type='ordered')
    part_c_gen = sim_plus.EntityGenerator(var_name='part_c', 
                                          steps_func=part_c_steps_func,
                                          env=env,
                                          arrival_type='inv_based', 
                                          cut_queue=True, 
                                          inv_level=3)
    
    # step 8: create all of the inventory locations in the simulation environment
    part_a_kanban = sim_plus.Kanban(var_name='part_a', env=env, 
#                                    kanban_attr=part_a.create_kanban_attrs(env.objs))
                                   kanban_attr=part_a.create_kanban_attrs(env._env_objs))
    part_b_kanban = sim_plus.Kanban(var_name='part_b', env=env, 
#                                    kanban_attr=part_b.create_kanban_attrs(env.objs))
                                   kanban_attr=part_b.create_kanban_attrs(env._env_objs))
    
    part_c_storage = sim_plus.Storage(var_name='part_c', env=env)
    scrap = sim_plus.Storage(var_name='scrap', env=env)
    
    # step 9: connect any entity boms and main_exits to its entity generator 
#     part_c_gen.bom = part_c.get_bom(env=env.objs)
    part_c_gen.bom = part_c.get_bom(env=env._env_objs)
    part_a_gen.main_exit = part_a_kanban
    part_b_gen.main_exit = part_b_kanban
    
    # step 10: activate the controlling components within the simulation environment
    assembler_shift.activate(process='work')
    technician_shift.activate(process='work')
    
    part_a_gen.activate(process='arrive')
    part_b_gen.activate(process='arrive')
    part_c_gen.activate(process='arrive')
    
    # step 11: execute the simulation
    env.run(till=runtime)